In [ ]:
from neo4j import GraphDatabase
from transformers import pipeline
from tqdm import tqdm
import pandas as pd
import torch

# Check GPU availability
device = 0 if torch.cuda.is_available() else -1
print(f"Using device: {'GPU' if device == 0 else 'CPU'}")

c:\Users\sachi\OneDrive\Desktop\Hackathon\MoodFlix-Smart-Movie-Recommender-System-mehul\KnowledgeBase\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: GPU


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli")
AutoTokenizer.from_pretrained("facebook/bart-large-mnli")

In [ ]:
df = pd.read_csv('eng.csv')

In [ ]:
# Load summarizer (optimized for GPU if available)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=device)

# Load zero-shot classifier (for mood classification)
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)

# Connect to Neo4j (replace with your credentials)
uri = "neo4j+s://fea8a723.databases.neo4j.io"
driver = GraphDatabase.driver(uri, auth=("neo4j", "Qp3U5o9HjMkHPuLjj9M4vL91doNcq3Hj4fGFpZV7-XI"))

Device set to use cuda:0
Device set to use cuda:0


In [ ]:
def summarize_text(text):
    input_length = len(text.split())
    max_len = min(50, input_length)  # avoid warning
    min_len = min(25, max_len-1) if max_len > 25 else 5
    summary = summarizer(text, max_length=max_len, min_length=min_len, do_sample=False)
    return summary[0]['summary_text']

def classify_runtime(text):
    return text

def create_graph(tx, title, runtime, year, rating):
    # Determine runtime category
    if runtime < 90:
        category = "Short"
    elif runtime <= 150:
        category = "Medium"
    else:
        category = "Large"
    
    tx.run("""
        MERGE (r:RuntimeCategory {name: $category})
        MERGE (m:Movie {title: $title})
        SET m.year = $year, 
            m.rating = $rating,
            m.runtime = $runtime
        MERGE (r)-[:CONTAINS]->(m)
        """, category=category, title=title, year=year, rating=rating, runtime=runtime)


In [ ]:
# Process movies with progress bar
with driver.session() as session:
    for _, row in tqdm(df.iterrows(), total=len(df)):
        title = row['Title']
        runtime = int(row['Runtime (Minutes)'])
        year = int(row['Year'])
        rating = float(row['Rating'])
        
        try:
            session.execute_write(create_graph, title, runtime, year, rating)
        except Exception as e:
            print(f"Error processing {title}: {e}")

driver.close()
print("✅ Runtime-Movie knowledge graph built successfully!")

 97%|█████████▋| 974/1000 [09:09<00:13,  1.90it/s]